In [1]:
import sys
sys.path.insert(0, "../")

In [2]:
import graph
import importlib
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import schedule as sch
import time
import topo
import typing
import utils
import vivaldi
import yaml

In [3]:
topo_file = "../samples/1e3h.yaml"
sc = topo.Scenario.from_dict(yaml.load(open(topo_file, "r").read(), Loader=yaml.Loader))

In [4]:
Coord3D = vivaldi.create_coordinate_class(3)
coords = dict()
for n in sc.topo.get_hosts():
    coords[n.uuid] = Coord3D.random_unit_vector()
coords_result = vivaldi.vivaldi_compute(sc.topo, coords, 0.1, 3000)

2021-11-22 23:06:01 Vancior-Redmi vivaldi.compute[951] DEBUG initial error: 68.813


In [5]:
for k, c in coords_result.items():
    print(k, c)
rasp1 = coords_result['rasp1']
rasp2 = coords_result['rasp2']
rasp3 = coords_result['rasp3']
# vm1 = coords_result['vm1']
# vm2 = coords_result['vm2']
# print(abs(rasp1-rasp2))
# print(abs(rasp1-rasp3))
# print(abs(rasp1-vm1))
# print(abs(vm1-vm2))

cloud1 (-7.053095203354364,-4.415008187251474,-16.768368096042785)
rasp1 (3.1655331600950714,2.1883574893059192,4.357026340112555)
rasp2 (1.515260894953106,1.455626102430396,5.691384881146762)
rasp3 (3.1561212893820425,0.5898296700396481,5.315991818785334)


In [6]:
with open("../cases/dag1.yaml") as f:
    graph_list: typing.List[graph.ExecutionGraph] = graph.ExecutionGraph.load_all(f)
print(len(graph_list))

10


In [7]:
def get_graph_domain(g: graph.ExecutionGraph) -> topo.Domain:
    domain_set = set()
    for s in g.get_sources():
        for d in sc.get_edge_domains():
            if d.find_host(s.domain_constraint["host"]) is not None:
                domain_set.add(d.name)
    assert len(domain_set) == 1
    return sc.find_domain(list(domain_set)[0])

In [17]:
sc.topo.clear_occupied()
result_list = [sch.SchedulingResult() for _ in graph_list]
op_pick_list = dict()
for g, r in zip(graph_list, result_list):
    op_coords = {v.uuid: Coord3D.random_unit_vector() for v in g.get_vertices()}
    movable = dict()
    for v in g.get_vertices():
        movable[v.uuid] = len(v.domain_constraint) == 0
        if not movable[v.uuid]:
            op_coords[v.uuid] = coords_result[v.domain_constraint['host']]
    op_result = vivaldi.constrained_balance(g, op_coords, movable, 0.001, 2000)
    edge_domain = get_graph_domain(g)
    cloud_domain = sc.get_cloud_domains()[0]
    for v in g.get_sources():
        host = edge_domain.find_host(v.domain_constraint['host'])
        assert host is not None
        r.assign(host.node.uuid, v.uuid)
        host.node.occupy(1)
    for v in g.get_sinks():
        host = cloud_domain.find_host(v.domain_constraint['host'])
        assert host is not None
        r.assign(host.node.uuid, v.uuid)
        host.node.occupy(1)
    for v in g.get_operators():
        op_pick_list[v.uuid] = op_result[v.uuid]

In [9]:
print(sum([g.number_of_vertices() for g in graph_list]))
print(len(op_pick_list))

99
55


In [10]:
class PickItem:
    def __init__(self, coord):
        self.coord = coord
        self.min_dist = None

op_pick_list = {k: PickItem(v) for k, v in op_pick_list.items()}

In [11]:
big_result = sch.SchedulingResult()

while True:
    while True:
        updated = False
        for domain in sc.get_edge_domains() + sc.get_cloud_domains():
            for host in domain.topo.get_hosts():
                if host.slots <= host.occupied or len(op_pick_list) == 0:
                    continue
                self_coord = coords_result[host.uuid]
                min_dist = 1e10
                min_op = None
                for op_id, op_item in op_pick_list.items():
                    dist = abs(op_item.coord - self_coord)
                    if dist < min_dist:
                        min_dist = dist
                        min_op = op_id
                    if op_item.min_dist is None or dist < op_item.min_dist:
                        op_item.min_dist = dist
                assert min_dist < 1e10 and min_op is not None
                big_result.assign(host.uuid, min_op)
                host.occupy(1)
                op_pick_list.pop(min_op)
                updated = True
        if not updated:
            break
    if len(op_pick_list) == 0:
        break
    for _, op_item in op_pick_list:
        op_item.min_dist = None


In [12]:
for idx, g in enumerate(graph_list):
    result_list[idx] = sch.SchedulingResult.merge(result_list[idx], big_result.extract(set([v.uuid for v in g.get_operators()])))

In [13]:
for r in result_list:
    print(r.assign_map)

{'g0-v0': 'rasp1', 'g0-v5': 'cloud1', 'g0-v7': 'cloud1', 'g0-v8': 'cloud1', 'g0-v1': 'rasp1', 'g0-v2': 'cloud1', 'g0-v3': 'cloud1', 'g0-v6': 'cloud1', 'g0-v4': 'cloud1'}
{'g1-v0': 'rasp1', 'g1-v7': 'cloud1', 'g1-v4': 'cloud1', 'g1-v5': 'cloud1', 'g1-v1': 'rasp1', 'g1-v3': 'rasp2', 'g1-v6': 'cloud1', 'g1-v2': 'rasp1'}
{'g2-v0': 'rasp3', 'g2-v2': 'cloud1', 'g2-v5': 'cloud1', 'g2-v8': 'cloud1', 'g2-v9': 'cloud1', 'g2-v6': 'cloud1', 'g2-v1': 'cloud1', 'g2-v4': 'cloud1', 'g2-v7': 'cloud1', 'g2-v3': 'rasp2'}
{'g3-v0': 'rasp1', 'g3-v10': 'cloud1', 'g3-v11': 'cloud1', 'g3-v2': 'cloud1', 'g3-v9': 'cloud1', 'g3-v8': 'cloud1', 'g3-v4': 'cloud1', 'g3-v7': 'cloud1', 'g3-v1': 'rasp2', 'g3-v6': 'cloud1', 'g3-v3': 'rasp1', 'g3-v5': 'cloud1'}
{'g4-v0': 'rasp3', 'g4-v3': 'cloud1', 'g4-v7': 'cloud1', 'g4-v8': 'cloud1', 'g4-v9': 'cloud1', 'g4-v1': 'cloud1', 'g4-v2': 'cloud1', 'g4-v6': 'cloud1', 'g4-v4': 'cloud1', 'g4-v5': 'cloud1'}
{'g5-v0': 'rasp1', 'g5-v10': 'cloud1', 'g5-v6': 'cloud1', 'g5-v8': 'cloud1

In [14]:
calculator = sch.LatencyCalculator(sc.topo)
for g, r in zip(graph_list, result_list):
    calculator.add_scheduled_graph(g, r)
lat, bp = calculator.compute_latency()
print(lat)
print(bp)

{'g0': 37.35632183908046, 'g1': 39.16685205784204, 'g2': 41.40911330049261, 'g3': 36.275238095238095, 'g4': 36.0, 'g5': 40.325073313782994, 'g6': 39.857549857549856, 'g7': 36.359060846560844, 'g8': 35.34615384615385, 'g9': 34.0}
{'g0': 0.0, 'g1': 0.0, 'g2': 0.0, 'g3': 0.0, 'g4': 0.0, 'g5': 0.0, 'g6': 0.0, 'g7': 0.0, 'g8': 0.0, 'g9': 0.0}


In [15]:
g = graph_list[0]
op_coords = {v.uuid: Coord3D.random_unit_vector() for v in g.get_vertices()}
movable = dict()
for v in g.get_vertices():
    movable[v.uuid] = len(v.domain_constraint) == 0
    if not movable[v.uuid]:
        op_coords[v.uuid] = coords_result[v.domain_constraint['host']]
op_result = vivaldi.constrained_balance(g, op_coords, movable, 0.001, 2000)
for v in g.get_vertices():
    print(v.uuid)
    for host in sc.topo.get_hosts():
        print(host.uuid, abs(op_result[v.uuid] - coords_result[host.uuid]))

g0-v0
cloud1 24.378414508982583
rasp1 0.0
rasp2 2.2451740579385504
rasp3 1.8641337295281717
g0-v1
cloud1 19.19784575055655
rasp1 5.180568770233503
rasp2 5.88512773417958
rasp3 5.864762058752839
g0-v2
cloud1 15.072024844314136
rasp1 9.306389697316748
rasp2 9.828513306693903
rasp3 9.870000959452021
g0-v3
cloud1 9.169843951328478
rasp1 15.208570592893013
rasp2 15.63433156304683
rasp3 15.70860525368117
g0-v4
cloud1 16.148307898879985
rasp1 8.230106652572461
rasp2 8.783646603847348
rasp3 8.814424581782086
g0-v5
cloud1 0.0
rasp1 24.378414508982583
rasp2 24.745132389764375
rasp3 24.839393233752762
g0-v6
cloud1 7.138599835502594
rasp1 17.23981471381755
rasp2 17.647208279236104
rasp3 17.7277066461338
g0-v7
cloud1 0.0
rasp1 24.378414508982583
rasp2 24.745132389764375
rasp3 24.839393233752762
g0-v8
cloud1 0.0
rasp1 24.378414508982583
rasp2 24.745132389764375
rasp3 24.839393233752762


In [16]:
sc.topo.clear_occupied()
flow_calculator = sch.LatencyCalculator(sc.topo)
flow_scheduler = sch.FlowScheduler(sc)
flow_result_list = flow_scheduler.schedule_multiple(graph_list)
for g, r in zip(graph_list, flow_result_list):
    assert r is not None
    flow_calculator.add_scheduled_graph(g, r)
lat, bp = flow_calculator.compute_latency()
print(lat)
print(bp)

{'g0': 38.241379310344826, 'g1': 34.454444332086155, 'g2': 37.52339901477833, 'g3': 35.905952380952385, 'g4': 37.17283950617284, 'g5': 35.538514173998045, 'g6': 37.35765160765161, 'g7': 35.276344797178126, 'g8': 39.0, 'g9': 33.0}
{'g0': 0.0, 'g1': 0.0, 'g2': 0.0, 'g3': 0.0, 'g4': 0.0, 'g5': 0.0, 'g6': 0.0, 'g7': 0.0, 'g8': 0.0, 'g9': 0.0}
